In [1]:
import numpy as np
import pandas as pd

In [179]:
import seaborn

In [8]:
hostm = pd.read_excel("data/Host_metabolome.xlsx", sheet_name=None)

In [145]:
biom = pd.read_excel("data/meta-metabolome.xlsx", sheet_name=None)

In [78]:
labels = pd.read_excel("data/Microbiome (1).xlsx", sheet_name=None)

In [56]:
enum_times = [i for i in biom.keys()]

nature of potential data errors

In [200]:
for t in enum_times:
    s = len(biom[t])
    f = 0
    fi = 0
    for i in range(s):
        curr = []
        for j in ['BR1', 'BR2', 'BR3']:
            if not np.isnan(biom[t][j][i]):
                curr.append(biom[t][j][i])
        if len(curr) < 3:
            f += 1
        cv = np.std(curr)/np.mean(curr)
        if cv > 0.5:
            fi += 1
    print("{}: nan values {}, too wide {}".format(t, f*100/s, fi*100/s))

3wk: nan values 53.160919540229884, too wide 33.91909814323608
4wkTC: nan values 49.58089332874038, too wide 26.145366861867036
4wkAB: nan values 43.582807333934475, too wide 26.59052199178439
6wkTC: nan values 58.15899581589958, too wide 32.75009509319133
6wkAB: nan values 50.84089894329513, too wide 27.429677035273105
8wkTC: nan values 54.90307867730901, too wide 35.034207525655646
8wkAB: nan values 44.47420634920635, too wide 27.36111111111111
10wkTC: nan values 44.4593895499224, too wide 28.20486290739783
10wkAB: nan values 55.44083684282027, too wide 32.237467735362046


almost 50% data missing a value, with about 30% being too inconsistant.

In [67]:
for j in ['BR1', 'BR2', 'BR3']:
    e = []
    for t in enum_times:
        s = len(biom[t])
        f = 0
        for i in range(s):
            if np.isnan(biom[t][j][i]):
                f += 1
        e.append(f*100/s)
    print("mean nan amount for {}: {}".format(j, np.mean(e)))
    
        

mean nan amount for BR1: 18.213078070565878
mean nan amount for BR2: 16.376306943138655
mean nan amount for BR3: 15.921951250890743


take `BR3` for now.

In [154]:
m3biom = biom.copy()
for t in enum_times:
    m3biom[t] = biom[t].copy(deep=True)
    m3biom[t].drop(columns= ['BR1', 'BR2'], inplace=True)
    m3biom[t].rename(columns = {'BR3':'value'}, inplace = True)
    m3biom[t].dropna(inplace=True)

In [171]:
m3biom['4wkTC']

,Metabolites,value
3,"2,6-Bis(2-benzimidazolyl)pyridine",7.023801e+03
4,Gly-Gly-Gly amide,8.303520e+03
6,2-Formyl-N-phenylhydrazinecarbothioamide,1.044986e+04
9,3-Chloro-5-methoxy-1-benzothiophene-2-carboxyl...,4.979506e+03
10,Matairesinoside,2.757464e+03
...,...,...
8703,"[7-(3,4-dihydroxyphenyl)-1-(4-hydroxyphenyl)he...",5.085722e+06
8704,Methacholine cation,5.300421e+06
8705,SFRNGVGSGVKKTSFRRAKQ/4,4.433277e+04
8706,2-[2-Oxo-2-(3-pyridinylamino)ethyl]benzoic acid,6.062787e+06



label data

assuming mutually exclusive

In [184]:
m3l1 = {}
for t in enum_times:
    m3l1[t] = labels['Phylum'][['index', t+'3']]
    m3l1[t] = m3l1[t][~(m3l1[t] == 0.0).any(axis=1)]

In [185]:
m3l1['4wkTC']

,index,4wkTC3
5,Fusobacteria,0.056529
9,Deferribacteres,0.113058
12,Proteobacteria,1.074053
14,Spirochaetes,0.056529
16,Epsilonbacteraeota,0.282646
17,Verrucomicrobia,0.056529
18,Bacteroidetes,21.368005
20,Tenericutes,0.169587
22,Patescibacteria,0.113058
26,Euryarchaeota,0.113058


In [186]:
m3l2 = {}
for t in enum_times:
    m3l2[t] = labels['Genus'][['index', t+'3']]
    m3l2[t] = m3l2[t][~(m3l2[t] == 0.0).any(axis=1)]

In [187]:
m3l2['4wkTC']

,index,4wkTC3
2,Ochrobactrum,0.090585
9,Peptococcus,0.181169
15,Lachnospiraceae-UCG-006,0.362339
16,Micrococcus,0.090585
29,Marmoricola,0.090585
...,...,...
471,Parasutterella,0.090585
483,Lactobacillus,0.996431
484,Ruminococcaceae-UCG-014,0.815262
486,Desulfovibrio,0.724677


In [195]:
m3l = {}
for t in enum_times:
    m3l[t] = pd.concat([m3l1[t], m3l2[t]])


In [197]:
m3l['4wkTC']

,index,4wkTC3
5,Fusobacteria,0.056529
9,Deferribacteres,0.113058
12,Proteobacteria,1.074053
14,Spirochaetes,0.056529
16,Epsilonbacteraeota,0.282646
...,...,...
471,Parasutterella,0.090585
483,Lactobacillus,0.996431
484,Ruminococcaceae-UCG-014,0.815262
486,Desulfovibrio,0.724677
